# Brainstorming Diferent Module Architectures

In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import EncoderDecoderModel, EncoderDecoderConfig, BertConfig
from transformers import GPT2Tokenizer, GPT2LMHeadModel, BertForMaskedLM, BertTokenizer
from transformers import T5Tokenizer
import torch.optim as optim
import concurrent.futures
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import os
from dotenv import load_dotenv

2023-02-17 01:19:21.426634: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-17 01:19:22.286292: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-17 01:19:24.156628: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-17 01:19:24.157005: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

### Data

In [2]:
topics = ['Art', 'Science', 'Technology']
keywords = ['Painting', 'Physics', 'Computer Science']
questions = ['What is you preferred hobbie?', 'What is a favorite discipline?',
             'Is python your programming language?']

## Data Preparation 

In [4]:
#pytorch
# cs, key, q = zip(*batch)
#     top_enc = tokenizer(topics, padding=True, truncation=True, return_tensors='pt')
#     key_enc = tokenizer(keywords, padding=True, truncation=True, return_tensors='pt')
#     q_enc = tokenizer(questions, padding=True, truncation=True, return_tensors='pt')
# print(top_enc)
# print(key_enc)
# print(q_enc)

In [5]:
# t5-base? t5-small?
# tokenizer = T5Tokenizer.from_pretrained('t5-small')

In [6]:
# !pip install sentencepiece

In [7]:
# tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# model = GPT2LMHeadModel.from_pretrained('gpt2')

In [8]:
# t_data = []
# for t, k in zip(topics, keywords):
#     prompt = f"Topic: {t} | Keyword: {k} | Question: "
#     t_data.append(prompt)
# print(t_data)

In [9]:

# i = []
# t = []
# for t, k, q in zip(topics, keywords, questions):
#     input_str = f"generate a question for topic {topic} and keyword {keyword}:"
#     i_tokens = tokenizer.tokenize(i_str)
#     i_ids = tokenizer.convert_tokens_to_ids(i_tokens)
#     i_ids = torch.tensor(i_ids)
#     t_str = f"{q} [SEP]"
#     t_tokens = tokenizer.tokenize(t_str)
#     t_ids = tokenizer.convert_tokens_to_ids(t_tokens)
#     t_ids = torch.tensor(t_ids)
#     i.append(i_ids)
#     t.append(t_ids)
# inputs = torch.nn.utils.rnn.pad_sequence(i, batch_first=True, padding_value=tokenizer.pad_token_id)
# targets = torch.nn.utils.rnn.pad_sequence(t, batch_first=True, padding_value=tokenizer.pad_token_id)
# attention_mask = inputs != tokenizer.pad_token_id
# print(inputs) 
# print(attention_mask) 
# print(targets)

# Models Options 

Considerations to build my model:

1. set of topic-keyword pairs 
2. prepare a set of training prompts that include the topic, keyword, and a placeholder for the question. 
3. fine-tune the GPT-2 model on the training prompts by minimizing the difference between the model's predicted next-token and the actual next-token. 
4. generate questions for each topic-keyword pair by providing the trained model with a prompt that includes the topic, keyword, and the question placeholder, and using the generate method to generate a new question based on the provided prompt.

## PyTorch and Hugging Face Transformer

sequence-to-sequence

In [34]:
# dataset class
class TopicKeywordDataset(Dataset):
    def __init__(self, topics, keywords, questions):
        self.topics = topics
        self.keywords = keywords
        self.questions = questions
        
    def __len__(self):
        return len(self.topics)
    
    def __getitem__(self, idx):
        topic = self.topics[idx]
        keyword = self.keywords[idx]
        question = self.questions[idx]
        return (topic, keyword, question)

In [35]:
def data_loadr(batch):
    topics, keywords, questions = zip(*batch)
    topics_enc = tokenizer(topics, padding=True, truncation=True, return_tensors='pt')
    keywords_enc = tokenizer(keywords, padding=True, truncation=True, return_tensors='pt')
    questions_enc = tokenizer(questions, padding=True, truncation=True, return_tensors='pt')
    return topics_enc, keywords_enc, questions_enc

In [36]:

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [37]:
dataset = TopicKeywordDataset(topics, keywords, questions)
data_loader = DataLoader(dataset, batch_size=3, collate_fn=data_loadr)




In [38]:
# model configuration and encoder-decoder model
encoder_config = BertConfig.from_pretrained('bert-base-uncased')
decoder_config = EncoderDecoderConfig.from_encoder_decoder_configs(encoder_config, encoder_config)
model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-uncased', 'bert-base-uncased', config=decoder_config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relations

In [39]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
# print(model)
# break
model.train()

for epoch in range(5):
    for batch in data_loader:
        topics_enc, keywords_enc, questions_enc = batch
        input_ids = torch.cat([topics_enc['input_ids'], keywords_enc['input_ids']], dim=1)
        # print(input_ids)
        output_ids = questions_enc['input_ids']
        attention_mask = torch.cat([topics_enc['attention_mask'], keywords_enc['attention_mask']], dim=1)
        # print(attention_mask)
        decoder_attention_mask = questions_enc['attention_mask']
        # print(decoder_attention_mask)
        loss = model(input_ids=input_ids, attention_mask=attention_mask, decoder_input_ids=output_ids, decoder_attention_mask=decoder_attention_mask).loss
        # loss.backward() #forward did not return a valid loss value
        optimizer.step()
        optimizer.zero_grad()

In [40]:
# evaluating a question generator?

model.eval()

# generated_questions = []
# for topic, keyword in zip(topics, keywords):
#     input_text = f"generate question for {topic} and {keyword}"
#     input_ids = tokenizer.encode(input_text, add_special_tokens=True, return_tensors='pt')
#     # input_ids = tokenizer.encode(f"generate question for {topic} and {keyword}", return_tensors='pt')
#     generated = model.generate(input_ids, max_length=50, do_sample=True, 
#                                decoder_start_token_id=tokenizer.encode(tokenizer.bos_token)[0])
#     question = tokenizer.decode(generated[0], skip_special_tokens=True)
#     generated_questions.append(question)

# generated_questions = []
# for topic, keyword in zip(topics, keywords):
#     input_ids = tokenizer.encode(f"generate question for {topic} and {keyword}", return_tensors='pt')
#     generated = model.generate(
#         input_ids, 
#         max_length=50, 
#         do_sample=True,
#         generation_config=EncoderDecoderGenerationConfig(bos_token_id=tokenizer.bos_token_id)
#     )
#     question = tokenizer.decode(generated[0], skip_special_tokens=True)
#     generated_questions.append(question)

generated_questions = []
for topic, keyword in zip(topics, keywords):
    input_text = f"generate question for {topic} and {keyword}"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    # generated_ids = model.generate(input_ids, max_length=50, do_sample=True, decoder_start_token_id=tokenizer.bos_token_id)
    generated = model.generate(input_ids, max_length=50, do_sample=True, 
                           decoder_start_token_id=model.config.decoder.pad_token_id)

    generated_question = tokenizer.decode(generated[0], skip_special_tokens=True)
    generated_questions.append(generated_question)





In [23]:
print(generated_questions)

['. body weres and and and and because because because are because because because because because as as as as as as as as as as as as as as as as as as as as as as as as as as as as as as as as', '. body body and utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility utility', '. representative representative representative representative representative representative of of silver silver silver silver bronze bronze silver silver silver bronze silver silver silver silver bronze bronze bronze silver silver silver silver gold silver silver silver silver silver silver silver silver silver silver silver silver silver silver silver silver silver silver']


### GPT-2

In [28]:
# pre-trained GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [29]:
train_data = []
for topic, keyword in zip(topics, keywords):
    prompt = f"Topic: {topic} | Keyword: {keyword} | Question: "
    train_data.append(prompt)

In [30]:
model.train()
for epoch in range(5):
    for prompt in train_data:
        input_ids = tokenizer.encode(prompt, return_tensors='pt')
        output = model(input_ids=input_ids, labels=input_ids)
        loss = output.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [31]:
# evaluating a question generator?
model.eval()
generated_questions = []
for topic, keyword in zip(topics, keywords):
    prompt = f"Topic: {topic} | Keyword: {keyword} | Question: "
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    generated = model.generate(input_ids, max_length=50, do_sample=True)
    question = tokenizer.decode(generated[0], skip_special_tokens=True)
    generated_questions.append(question)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [32]:
print(generated_questions)


['Topic: Art | Keyword: Painting | Question: __________________\n\nI hope that your art is well presented. Let us know if you can use some.', "Topic: Science | Keyword: Physics | Question: 【How does a planet in the solar system work】 | Question: (what's the physics aspect of a particular planet) | Question: (where is the planet, the planet's orbit", 'Topic: Technology | Keyword: Computer Science | Question: -------------- next part -------------- An HTML attachment was scrubbed... URL: <http://mail.reuters.com/id/pdp-sydna.html>']


## BERT

In [18]:
from torchtext.data import Field

ImportError: cannot import name 'Field' from 'torchtext.data' (/home/dinis/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/torchtext/data/__init__.py)

In [8]:

class TopicKeywordDataset(Dataset):
    def __init__(self, topics, keywords, questions):
        self.topics = topics
        self.keywords = keywords
        self.questions = questions
        
    def __len__(self):
        return len(self.topics)
    
    def __getitem__(self, idx):
        topic = self.topics[idx]
        keyword = self.keywords[idx]
        question = self.questions[idx]
        return (topic, keyword, question)


In [11]:
def data_loadr_2(batch):
    topics, keywords, questions = zip(*batch)
    inputs = []
    targets = []
    max_input_len = max(len(tokenizer.tokenize(f"generate a question for topic {topic} and keyword {keyword}:")) + len(tokenizer.tokenize(question)) + 1 for topic, keyword, question in batch) # plus 1 for [SEP] token
    
    # for topic, keyword, question in zip(topics, keywords, questions):
    for topic, keyword, question in batch:
        input_str = f"generate a question for topic {topic} and keyword {keyword}: {question} [SEP]"
        input_tokens = tokenizer.tokenize(input_str)
        input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
        inputs.append(input_ids)
        # print("Input shape:", input_ids.size())  
        target_tokens = input_tokens[1:-1]  # exclude [CLS] and [SEP] tokens
        target_ids = tokenizer.convert_tokens_to_ids(target_tokens)
        # target_ids = tokenizer.convert_tokens_to_ids(target_tokens)
        target_ids += [-100] * (max_input_len - len(target_ids))  # pad with -100
        target_ids = torch.tensor(target_ids)
        targets.append(target_ids)
        # print("Target shape:", target_ids.size())
        
    batch_size = len(batch)
    
    # pad the inputs
    inputs = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=True, padding_value=tokenizer.pad_token_id)
    
    # targets_flat = [t for target in targets for t in target]
#     targets_batch = targets_flat[:batch_size]
    # pad the targets
    targets = torch.nn.utils.rnn.pad_sequence(targets, batch_first=True, padding_value=-100)
    # targets = torch.nn.functional.pad(targets, (0, max_target_len - targets.shape[1]), value=tokenizer.pad_token_id)
    
    
    # for i, (input_seq, target_seq) in enumerate(zip(inputs, targets)):
    #     print(f"Batch {i}: input shape={input_seq.shape}, target shape={target_seq.shape}")  
    
    attention_mask = inputs != tokenizer.pad_token_id
    return inputs, attention_mask, targets


In [10]:
# def data_loadr_2(batch):
#     inputs = [torch.tensor(item[0], dtype=torch.long) for item in batch]
#     targets = [torch.tensor(item[1], dtype=torch.float) for item in batch]
    
#     # Print statements to see how the inputs and targets are being processed
#     print("inputs before padding:", inputs)
#     print("targets before padding:", targets)
    
#     inputs = pad_sequence(inputs, batch_first=True)
#     targets = pad_sequence(targets, batch_first=True)
    
#     # Print statements to see how the inputs and targets look after padding
#     print("inputs after padding:", inputs)
#     print("targets after padding:", targets)
    
#     return inputs, targets


In [5]:
 
model = BertForMaskedLM.from_pretrained('bert-base-uncased')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [22]:
question = questions[0]
topic = topics[0]
bert_tokenized_sequence = tokenizer.tokenize(question)

In [24]:
max_len_bert = 0
max_len_roberta = 0

for question in questions:

    # `[CLS]`  `[SEP]` tokens.
    input_ids_bert = tokenizer.encode(question, add_special_tokens=True)

    
    max_len_bert = max(max_len_bert, len(input_ids_bert))

    
print('Max sentence length BERT: ', max_len_bert)

Max sentence length BERT:  10


In [28]:
from collections import Counter

topic_counts = Counter(topics)
topic_to_label = {topic: i for i, (topic, _) in enumerate(topic_counts.most_common())}

labels = torch.tensor([topic_to_label[topic] for topic in topics])


In [ ]:
dataset = TopicKeywordDataset(topics, keywords, questions)
data_loader_2 = DataLoader(dataset, batch_size=3, collate_fn=data_loadr_2, drop_last=True)
for i, batch in enumerate(data_loader_2):
    print(f"Batch {i}: {batch}")

In [29]:
bert_input_ids = []
bert_attention_masks = []
question_ids = []
counter = 0

for question in questions:
    bert_encoded_dict = tokenizer.encode_plus(
                        question,                      # Sentence to encode.
                        add_special_tokens = False, # Add '[CLS]' and '[SEP]'
                        max_length = 10,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt', )# Return pytorch tensors.
    bert_input_ids.append(bert_encoded_dict['input_ids'])
    bert_attention_masks.append(bert_encoded_dict['attention_mask'])
    
    
    question_ids.append(counter)
    counter  = counter + 1
    
bert_input_ids = torch.cat(bert_input_ids, dim=0)
bert_attention_masks = torch.cat(bert_attention_masks, dim=0)

# labels = torch.tensor(topics)
question_ids = torch.tensor(question_ids)

print('Original: ', question)
print('Token IDs BERT:', bert_input_ids[1])

Original:  Is python your programming language?
Token IDs BERT: tensor([2054, 2003, 1037, 5440, 9009, 1029,    0,    0,    0,    0])


/home/dinis/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [30]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
model.train()
for epoch in range(5):
    for batch in data_loader_2:
        inputs, attention_mask, targets = batch
        print(inputs)
        print(targets)
        outputs = model(inputs, attention_mask=attention_mask, labels=targets)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


NameError: name 'data_loader_2' is not defined

In [63]:
# optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
# model.train()
# for epoch in range(5):
#     for batch in data_loader_2:
#         try:
#             inputs, attention_mask, targets = batch
#             outputs = model(inputs, attention_mask=attention_mask, labels=targets)
#             loss = outputs.loss
#             loss.backward()
#             optimizer.step()
#             optimizer.zero_grad()
#         except Exception as e:
#             print("Error occurred:", e)
#             print("Inputs:", inputs)
#             print("Targets:", targets)


In [ ]:

# question generation
model.eval()
generated_questions = []
for topic, keyword in zip(topics, keywords):
    input_str = f"generate a question for topic {topic} and keyword {keyword}:"
    input_tokens = tokenizer.tokenize(input_str)
    input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
    input_ids = torch.tensor(input_ids).unsqueeze(0)
    generated = model.generate(input_ids, max_length=50, do_sample=True)
    question = tokenizer.decode(generated[0], skip_special_tokens=True)
    generated_questions.append(question)

print(generated_questions)


## Paralleling the models with a final estimator

with a RandomForest Final Estimator

In [6]:

# input_data = ...

# def preprocess_data(input_data):
#     preprocessed_data = ...
#     return preprocessed_data

# def run_gpt2_model(input_data):
#     gpt2_results = ...
#     return gpt2_results

# def run_pytorch_model(input_data):
#     pytorch_results = ...
#     return pytorch_results

# def run_bert_model(input_data):
#     bert_results = ...
#     return bert_results


In [8]:
final_estimator = RandomForestClassifier()


In [9]:
# pipeline = Pipeline([
#     ('preprocessing', preprocess_data),
#     ('modeling', concurrent.futures.ThreadPoolExecutor(max_workers=3)),
#     ('gpt2_model', run_gpt2_model),
#     ('pytorch_model', run_pytorch_model),
#     ('bert_model', run_bert_model),
#     ('final_estimator', final_estimator)
# ])
# pipeline

Pipeline(steps=[('preprocessing', <function preprocess_data at 0x7f10589d4a60>),
                ('modeling',
                 <concurrent.futures.thread.ThreadPoolExecutor object at 0x7f10507e2770>),
                ('gpt2_model', <function run_gpt2_model at 0x7f1050b8bf40>),
                ('pytorch_model',
                 <function run_pytorch_model at 0x7f1050bcc040>),
                ('bert_model', <function run_bert_model at 0x7f1050bcc0d0>),
                ('final_estimator', RandomForestClassifier())])

In [14]:
def preprocess_data(topics, keywords):
    train_data = []
    for topic, keyword in zip(topics, keywords):
        prompt = f"Topic: {topic} | Keyword: {keyword} | Question: "
        train_data.append(prompt)
    return train_data

In [15]:
def generate_questions(train_data):
    model.train()
    for epoch in range(5):
        for prompt in train_data:
            input_ids = tokenizer.encode(prompt, return_tensors='pt')
            output = model(input_ids=input_ids, labels=input_ids)
            loss = output.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
    model.eval()
    generated_questions = []
    for topic, keyword in zip(topics, keywords):
        prompt = f"Topic: {topic} | Keyword: {keyword} | Question: "
        input_ids = tokenizer.encode(prompt, return_tensors='pt')
        generated = model.generate(input_ids, max_length=50, do_sample=True)
        question = tokenizer.decode(generated[0], skip_special_tokens=True)
        generated_questions.append(question)
    return generated_questions


gpt_pipeline = Pipeline([
    ('preprocessing', preprocess_data),
    ('question_generation', generate_questions)
])

In [16]:
gpt_pipeline

Pipeline(steps=[('preprocessing', <function preprocess_data at 0x7f104f2bc700>),
                ('question_generation',
                 <function generate_questions at 0x7f104f2bc4c0>)])

In [ ]:

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

requesting open AI to start a question